In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim

In [2]:
training_data = [
        ("Veinte paginas".lower().split(), "Spanish"),
        ("I will visit the library".lower().split(), "English"),
        ("I am reading a book".lower().split(), "English"),
        ("This is my favourite chapter".lower().split(), "English"),
        ("Estoy en la biblioteca".lower().split(), "Spanish"),
        ("Tengo un libro".lower().split(), "Spanish")
        ]

test_data = [
        ("Estoy leyendo".lower().split(), "Spanish"),
        ("This is not my favourite book".lower().split(), "English")
        ]

In [3]:
word_dict = {}
i = 0
for words,language in training_data + test_data:
    for word in words:
        if word not in word_dict:
            word_dict[word]=i
            i+=1
print(word_dict)


{'veinte': 0, 'paginas': 1, 'i': 2, 'will': 3, 'visit': 4, 'the': 5, 'library': 6, 'am': 7, 'reading': 8, 'a': 9, 'book': 10, 'this': 11, 'is': 12, 'my': 13, 'favourite': 14, 'chapter': 15, 'estoy': 16, 'en': 17, 'la': 18, 'biblioteca': 19, 'tengo': 20, 'un': 21, 'libro': 22, 'leyendo': 23, 'not': 24}


Setting up the classifier

In [10]:
corpus_size = len(word_dict)
languages = 2
label_index = {"Spanish": 0, "English": 1}
class BagofWordsClassifier(nn.Module):
    
    def __init__(self,languages,corpus_size):
        super(BagofWordsClassifier,self).__init__()
        self.linear = nn.Linear(corpus_size,languages)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec),dim=1)

In [11]:
def make_bow_vector(sentence, word_index):
    word_vec = torch.zeros(corpus_size)
    for word in sentence:
        word_vec[word_dict[word]] += 1
    return word_vec.view(1, -1)

def make_target(label, label_index):
    return torch.LongTensor([label_index[label]])


In [14]:
# Stochatic Gradient descent(SGD)
model = BagofWordsClassifier(languages, corpus_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [17]:
# Training Classifier
for epoch in range(110):
    for sentence, label in training_data:
        model.zero_grad()
        bow_vec = make_bow_vector(sentence, word_dict)
        target = make_target(label, label_index)
        
        log_probs = model(bow_vec)
        
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
    if epoch%10==0:
        print('Epoch: ',str(epoch+1),' Loss: ' + str(loss.item()))

Epoch:  1  Loss: 0.5838678479194641
Epoch:  11  Loss: 0.12294137477874756
Epoch:  21  Loss: 0.06453198194503784
Epoch:  31  Loss: 0.04350188747048378
Epoch:  41  Loss: 0.03276314213871956
Epoch:  51  Loss: 0.02626236155629158
Epoch:  61  Loss: 0.02190852351486683
Epoch:  71  Loss: 0.018790453672409058
Epoch:  81  Loss: 0.01644790731370449
Epoch:  91  Loss: 0.014624071307480335
Epoch:  101  Loss: 0.013163803145289421


In [20]:
!git status
!git add *

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .ipynb_checkpoints/Bag of Word-Classifier-checkpoint.ipynb
	new file:   Bag of Word-Classifier.ipynb
	modified:   NN.ipynb

